In [0]:
import matplotlib.pyplot as plt
import numpy as np
from termcolor import colored
file_data   = "drive/My Drive/Classroom/Machine Learning (2) 2020-1\
/class-MachineLearning/assignment09/mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)
idx_label = [[] for i in range(10)]
for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)
    
    list_label[count]       = label
    list_image[:, count]    = im_vector
    idx_label[int(label)].append(count)
    count += 1

#
# plot first 150 images out of 10,000 with their labels
#
'''
f1 = plt.figure(1)

for i in range(150):

    label       = list_label[i]
    im_vector   = list_image[:, i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(10, 15, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

'''
#plt.show()

#
# plot the average image of all the images for each digit
#
f2 = plt.figure(2)

im_average  = np.zeros((size_row * size_col, 10), dtype=float)
im_count    = np.zeros(10, dtype=int)

for i in range(num_image):

    im_average[:, list_label[i]] += list_image[:, i]
    im_count[list_label[i]] += 1

for i in range(10):

    im_average[:, i] /= im_count[i]

    plt.subplot(2, 5, i+1)
    plt.title(i)
    plt.imshow(im_average[:,i].reshape((size_row, size_col)), cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

plt.show()

In [0]:
'''
Config
'''

learning_rate = 1e-1
#input_data=np.vstack((bias, list_image))

one_hot=np.zeros((10, num_image))
for i in range(num_image) :
    one_hot[list_label[i]][i]=1

train_image = list_image[:, :6000]
train_label = one_hot[:, :6000]

test_image = list_image[:, 6000:]
test_label = one_hot[:, 6000:]

num_train = train_image.shape[1]
num_test = test_image.shape[1]

#input_data= np.vstack(((np.full((1, list_image.shape[1]), bias0)), list_image))
layer1=np.empty((196, num_train), dtype=float)
layer2=np.empty((49, num_train), dtype=float)
layer3=np.empty((10, num_train), dtype=float)
layers=[train_image, layer1, layer2, layer3]

weight0=np.random.normal(0., 1, (196, size_row * size_col))
weight1=np.random.normal(0., 1, (49, 196))
weight2=np.random.normal(0., 1, (10, 49))
weights=[weight0, weight1, weight2]

bias0, bias1, bias2 = 1, 1, 1
biases=[bias0, bias1, bias2]

train_loss=[]
train_accuracy=[]

test_loss=[]
test_accuracy=[]

print(num_train)
print(num_test)

In [0]:
#a0=np.zeros(())
def sigmoid(x) :
    return 1/(1+np.exp(-x))

def fw_propagation() :
    #a_list=[]
    #input_data = np.vstack(((np.full((1, list_image.shape[1]), bias0)), list_image))
    for i in range(len(weights)) :
        layers[i+1]=sigmoid(weights[i] @ layers[i] + biases[i])
        # h0=weight0 @ input_data
        #a0=np.vstack(((np.full((1, a0.shape[1]), bias1)), a0))
        #a0=np.vstack((np.ones((1, a0.shape[1])), a0))
        
    #     h1=weight1 @ a0
    #     a1=sigmoid(h1)
    #     #a1=np.vstack((np.ones((1, a1.shape[1])), a1))
    #     a1=np.vstack(((np.full((1, a1.shape[1]), bias2)), a1))
    #     h2=weight2 @ a1
    #     a2=sigmoid(h2)
def predict() :
    for i in range(len(weights)) :
        if i==0 : 
            output = sigmoid(weights[i] @ test_image + biases[i])
        else : 
            output = sigmoid(weights[i] @ output + biases[i])

    return output
                         
    #     a_list=[input_data, a0, a1, a2]
    # return a_list
#fw_propagation()
# print(a0.shape)
# print('-'*30)
# print(a1.shape)
# print('-'*30)
# print(a2.shape)
# print('-'*30)

In [0]:
'''
Back propagation
'''
def bw_propagation() :
    # global weight0
    # global weight1
    # global weight2
    # global bias0
    # global bias1
    # global bias2
    #depth = len(weights)-1  # depth = 2
    dev_h=layers[-1] - train_label
    for i in range(len(weights)-1, -1, -1) :
        dev_w=dev_h @ layers[i].T
        dev_bias=np.sum(dev_h, axis=0)
        dev_h=weights[i].T @ dev_h * layers[i] * (1-layers[i])

        weights[i] -= learning_rate * dev_w / num_train
        biases[i] -= learning_rate * np.sum(dev_bias) / num_train

    # c0=layers[3]-one_hot
    # d0=c0 @ layers[2].T
    # b0=np.sum(c0, axis=0)

    # c1=(weights[2].T @ c0 * layers[2] * (1-layers[2]))
    # d1=c1 @ layers[1].T
    # b1=np.sum(c1, axis=0)

    # c2=(weights[1].T @ c1 * layers[1] * (1-layers[1]))
    # d2=c2 @ layers[0].T
    # b2=np.sum(c2, axis=0)

    # weight2-=learning_rate * d0 / input_data.shape[1]
    # weight1-=learning_rate * d1 / input_data.shape[1]
    # weight0-=learning_rate * d2 / input_data.shape[1]

    # bias2-=learning_rate * np.sum(b0) / input_data.shape[1]
    # bias1-=learning_rate * np.sum(b1) / input_data.shape[1]
    # bias0-=learning_rate * np.sum(b2) / input_data.shape[1]

def loss(output, label) :
    return np.mean(np.sum(-label*np.log(output) - (1-label)*np.log(1-output), axis=0))
def accuracy(output, label):
  return (output == label.argmax(axis = 0)).mean()*100

In [0]:
for i in range(20000) :
    # input_data, a0, a1, a2 = fw_propagation()
    fw_propagation()
    bw_propagation()
    p = predict()
    train_loss.append(loss(layers[-1], train_label))
    test_loss.append(loss(p, test_label))
    train_accuracy.append(accuracy(layers[-1].argmax(axis=0), train_label))
    test_accuracy.append(accuracy(p.argmax(axis=0), test_label))
    #l_list.append(l)
    # acc=accuracy(a2.argmax(axis=0), one_hot)
    # acc_list.append(acc)

# 1. Plot the loss curve

---
\
     
* train loss는 파란색, test loss는 빨간색으로 plot했습니다.

In [0]:
'''
Visualize Loss
'''
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='blue', label='train')
plt.plot(test_loss, color='red', label='test')
plt.xlabel("iteration")
plt.ylabel("cost")
plt.legend(loc='upper right')
plt.title("Loss")
plt.show()

# 2. Plot the accuracy curve

---
\
     
* train accuracy는 파란색, test accuracy는 빨간색으로 plot했습니다.

In [0]:
'''
Visualize accuracy
'''
plt.figure(figsize=(10, 7))
plt.plot(train_accuracy, color='blue', label='train')
plt.plot(test_accuracy, color='red', label='test')
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.legend(loc='upper right')
plt.title("Accuracy")
plt.show()

# 3. Plot the accuracy value

---
\
     
* train accuracy는 파란색, test accuracy는 빨간색으로 print했습니다.

In [0]:
print("<Final accuracy>")
print('-'*50)
train_final='train accuracy : '+str(train_accuracy[-1]) + '%'
test_filnal='test accuracy : '+str(test_accuracy[-1]) + '%'
print(colored(train_final, 'red'))
print(colored(test_final, 'green'))
print('-'*50)

# 4. Plot the classification example

---
\
     
* 

In [0]:
#plt.figure(figsize=(15,3))
pred = p.argmax(axis=0)
label = test_label.argmax(axis=0)

count=1
print("<Correct>")
print("-"*40)
for i in range(len(label)) :
    if pred[i] == label[i] :
        plt.subplot(2, 5, count)
        plt.title(pred[i])
        plt.imshow(test_image[:, i], cmap='Greys', interpolation='None')
        frame = gca()
        frame.axes.get_xaxis().set_visible(False)
        frame.axes.get_yaxis().set_visible(False)
        count+=1
    if count >10 :
        break
plt.show()

count=1
print("<Wrong>")
print("-"*40)
for i in range(len(label)) :
    if pred[i] != label[i] :
        plt.subplot(2, 5, count)
        plt.title(pred[i])
        plt.imshow(test_image[:, i], cmap='Greys', interpolation='None')
        frame = gca()
        frame.axes.get_xaxis().set_visible(False)
        frame.axes.get_yaxis().set_visible(False)
        count+=1
    if count >10 :
        break
plt.show()

In [0]:
'''
Visualize average image data with the same label
'''
print("Correct")
for i in range(10) :
    plt.subplot(2, 5, i+1)
    plt.title('label ' + str(i))
    plt.imshow(avg_matrix[i], cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)